In [1]:
import dataiku
from dataiku import pandasutils as pdu
import pandas as pd

In [2]:
client = dataiku.api_client()

In [3]:
project = client.get_project("DSPIPELINEFRAUD")

In [10]:
project_metadata = project.get_settings()

In [11]:
project_metadata

In [0]:
# list_jupyter_notebooks
# list_recipes
# list_running_notebooks